# Notebook for training the YOLOv8 model

# Train the model

In [2]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="config.yaml", epochs=10)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

#results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
#path = model.export(format="onnx")  # export the model to ONNX format

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_con

train: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/train/labels... 375 images, 0 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<00:00, 3671.05it/s]

train: New cache created: /home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/train/labels.cache



val: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/val/labels... 94 images, 0 backgrounds, 0 corrupt: 100%|██████████| 94/94 [00:00<00:00, 3500.34it/s]

val: New cache created: /home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/val/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 


...


10 epochs completed in 0.342 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.22s/it]


                   all         94        519      0.862      0.872      0.922      0.801
                   nut         94        144      0.736       0.84      0.851       0.67
           screw-small         94        148      0.885       0.73      0.902      0.756
                washer         94         72      0.969      0.958      0.979      0.887
           screw-large         94         83       0.95      0.917       0.96      0.871
              assembly         94         72      0.772      0.917      0.918      0.823
Speed: 1.4ms preprocess, 53.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/val/labels.cache... 94 images, 0 backgrounds, 0 corrupt: 100%|██████████| 94/94 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:06<00:00,  1.12s/it]


                   all         94        519      0.862      0.872      0.922      0.801
                   nut         94        144      0.736       0.84      0.851       0.67
           screw-small         94        148      0.885       0.73      0.902      0.756
                washer         94         72      0.969      0.958      0.979      0.887
           screw-large         94         83       0.95      0.917       0.96      0.871
              assembly         94         72      0.772      0.917      0.918      0.823
Speed: 1.4ms preprocess, 53.4ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train2


In [5]:
model.val()

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)


val: Scanning /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw/labels... 0 images, 9 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 9441.90it/s]

val: WARNING ⚠️ No labels found in /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw/labels.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw/labels.cache
WARNING ⚠️ No labels found in /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]

                   all          9          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 2.3ms preprocess, 106.5ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/train3


AttributeError: 'DetMetrics' object has no attribute 'curves_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass

# Inference

In [1]:
from pathlib import Path
from ultralytics import YOLO

def yolo_inference(source, weights):
    source_path = Path(source).absolute()
    weights_path = Path(weights).absolute()

    # Create YOLOv8 object
    yolo = YOLO(weights_path) #, imgsz=640)  # Adjust imgsz as needed

    yolo.val()
    # Run inference
    #results = yolo(source_path)

    # Display or process results as needed
    #print(results)

if __name__ == "__main__":
    source_path = "/home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/raw"
    weights_path = "/home/jetracer/Documents/3d_mai/notebooks/yolo_model/runs/detect/train/weights/best.pt"

    yolo_inference(source=source_path, weights=weights_path)


Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/test/labels... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 559.32it/s]

val: New cache created: /home/jetracer/Documents/3d_mai/data/real_images/2_real_imgs/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


                   all          9         18          1      0.302      0.523      0.278
                   nut          9          9          1          0      0.138     0.0482
           screw-small          9          9          1      0.605      0.908      0.508
Speed: 0.9ms preprocess, 55.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val2
